In [1]:
!git clone https://github.com/NichoJen/MultiNER.git

Cloning into 'MultiNER'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 53 (delta 26), reused 35 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 11.28 KiB | 5.64 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [2]:
!pip install wandb -qU
!pip install datasets transformers
!pip install evaluate
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=04d17dcfcd7e8eca2039ccfb3e2fe0177c6994bd90d6c3bccd08858f

In [3]:
# Insert the directory
import sys
sys.path.insert(0,"/content/MultiNER")

In [4]:
!mkdir models

In [5]:
!mkdir models\nl

In [6]:
# log into wandb
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


## Load Wiki-Neural dataset

In [8]:
from datasets import load_dataset
from transformers import XLMRobertaTokenizerFast
from transformers import XLMRobertaModel
import torch
from torch.utils.data import DataLoader
from MultiNER.utils import tokenize_and_align_labels
from MultiNER.XLMRobertaLinearEntityTagger import XLMRobertaLinearEntityTagger
from MultiNER.XLMRobertaLinearEntityTagger import train_model

In [9]:
# load dutch (nl) train, val and test data
wiki_neural_train = load_dataset("Babelscape/wikineural", split="train_nl")
wiki_neural_val = load_dataset("Babelscape/wikineural", split="val_nl")
wiki_neural_test = load_dataset("Babelscape/wikineural", split="test_nl")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test_de split: 0 examples [00:00, ? examples/s]

Generating test_en split: 0 examples [00:00, ? examples/s]

Generating test_es split: 0 examples [00:00, ? examples/s]

Generating test_fr split: 0 examples [00:00, ? examples/s]

...

Generating val_it split: 0 examples [00:00, ? examples/s]

Generating val_nl split: 0 examples [00:00, ? examples/s]

Generating val_pl split: 0 examples [00:00, ? examples/s]

Generating val_pt split: 0 examples [00:00, ? examples/s]

Generating val_ru split: 0 examples [00:00, ? examples/s]

In [10]:
print("wiki-neural-nl-train: ", wiki_neural_train)
print("wiki-neural-nl-val: ", wiki_neural_val)
print("wiki-neural-nl-test: ", wiki_neural_test)

wiki-neural-nl-train:  Dataset({
    features: ['tokens', 'ner_tags', 'lang'],
    num_rows: 83680
})
wiki-neural-nl-val:  Dataset({
    features: ['tokens', 'ner_tags', 'lang'],
    num_rows: 10460
})
wiki-neural-nl-test:  Dataset({
    features: ['tokens', 'ner_tags', 'lang'],
    num_rows: 10547
})


## Load XLM-Roberta tokenizer

In [11]:
#load tokenizer
XLMRoberta_tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

## Process Dataset and make dataloader


In [12]:
# dataloader hyperparameters
BATCH_SIZE = 64
HF_SHUFFLE_SEED = 42

In [13]:
# shuffle training set
wiki_neural_train = wiki_neural_train.shuffle(seed=HF_SHUFFLE_SEED)

In [15]:
# tokenize and align labels
wiki_neural_train = wiki_neural_train.map(tokenize_and_align_labels, batched=True, batch_size=BATCH_SIZE, fn_kwargs={"tokenizer": XLMRoberta_tokenizer})
wiki_neural_val = wiki_neural_val.map(tokenize_and_align_labels, batched=True, batch_size=BATCH_SIZE, fn_kwargs={"tokenizer": XLMRoberta_tokenizer})
wiki_neural_test = wiki_neural_test.map(tokenize_and_align_labels, batched=True, batch_size=BATCH_SIZE, fn_kwargs={"tokenizer": XLMRoberta_tokenizer})

Map:   0%|          | 0/83680 [00:00<?, ? examples/s]

Map:   0%|          | 0/10460 [00:00<?, ? examples/s]

Map:   0%|          | 0/10547 [00:00<?, ? examples/s]

In [16]:
# set format
wiki_neural_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
wiki_neural_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
wiki_neural_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [17]:
# define dataloaders for train dev and test set
train_loader = DataLoader(wiki_neural_train, batch_size=BATCH_SIZE, pin_memory=True)
val_loader = DataLoader(wiki_neural_val, batch_size=BATCH_SIZE, pin_memory=True)
test_loader = DataLoader(wiki_neural_test, batch_size=BATCH_SIZE, pin_memory=True)

## Train and evaluate model

In [18]:
# training hyperparameters
NUM_CLASSES = 9
TORCH_MANUAL_SEED = 42
LR = 1e-3
EPOCHS = 5

In [19]:
# check if gpu available
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {DEVICE}")

Using cuda


In [20]:
# load XLMRoberta model
xlm_roberta_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [21]:
# build linear tagger model
torch.manual_seed(TORCH_MANUAL_SEED)
NER_tagger = XLMRobertaLinearEntityTagger(num_classes=NUM_CLASSES, xlm_roberta_model=xlm_roberta_model)
NER_tagger.to(DEVICE)

XLMRobertaLinearEntityTagger(
  (xlm_roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [22]:
train_model(model=NER_tagger,
               lr=LR,
               epochs=EPOCHS,
               batch_size=BATCH_SIZE,
               train_loader=train_loader,
               val_loader=val_loader,
               test_loader=test_loader,
            device=DEVICE,
            project_name="xlm-Roberta-wiki-neural-eng",
            model_save_path="modelsnl/xlm_roberta_wiki_neural_nl",
            wandb_name=None,
            wandb_notes="xlm_roberta_wiki_neural_nl_2")

wandb: Currently logged in as: n1ch0 (n11ch00). Use `wandb login --relogin` to force relogin


epoch:  0


100%|██████████| 1308/1308 [01:36<00:00, 13.62it/s]


evaluating on train set


100%|██████████| 1308/1308 [02:28<00:00,  8.81it/s]


...

evaluating on train set


100%|██████████| 1308/1308 [02:27<00:00,  8.86it/s]


epoch loss: 0.05350099352618469
evaluating on val set


100%|██████████| 165/165 [00:19<00:00,  8.61it/s]


overall_precision,▁▄▆▇█
train/LOC/f1,▁▁▅▅▆▆▇▇██
train/LOC/number,▁▁▁▁▁▁▁▁▁▁
train/LOC/precision,▁▁▅▅▆▆▇▇██
train/LOC/recall,▁▁▅▅▇▇▇▇██
train/MISC/f1,▁▁▅▅▇▇████
train/MISC/number,▁▁▁▁▁▁▁▁▁▁
train/MISC/precision,▁▁▅▅▆▆▇▇██
train/MISC/recall,▁▁▅▅▇▇████
train/ORG/f1,▁▁▅▅▇▇▇▇██
train/ORG/number,▁▁▁▁▁▁▁▁▁▁


In [ ]:
from google.colab import runtime
runtime.unassign()